In [1]:
import re
import sys
sys.path.append("./")
import warnings
warnings.filterwarnings("ignore",category=FutureWarning)
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgbm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,scale
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor 
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans

from single_model import *


train = pd.read_csv("./train_with_ido.csv")
train = train.dropna(subset=["賃料"])
train = train.drop(20926)
train = train.drop(20231)
train = train.drop(20427)
train = train.drop(5775)
train = train.drop("Unnamed: 0",axis=1)
train_x = train.drop("賃料",axis=1)
train_y = train["賃料"]
test = pd.read_csv("./test_with_ido.csv")
test = test.drop("Unnamed: 0",axis=1)

/Users/kakeru/PycharmProjects/main_env/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
comment = """knnを落としてtrian 0nehotを追加"""
check_model(comment,train_x,train_y)

[21:26:46] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:27:11] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:27:54] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:29:10] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:29:39] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:30:20] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:31:37] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:32:02] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:32:37] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

In [2]:
commit(train_x,train_y,test,"submit_13",seeds=[7777,8888,5555,1111,1997])

[22:11:47] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:12:19] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:13:05] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:14:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:15:16] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:16:02] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:17:40] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:18:12] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:18:59] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

In [7]:
d = [[1,1,1,1,1,1],
     [2,2,2,2,2,2],
     [3,3,3,3,3,3]]
print (np.mean(d,axis=0))

[2. 2. 2. 2. 2. 2.]


In [3]:
m = pre_checker(train_x,train_y)
hoge = m.predict(train)

[10:01:47] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [4]:
hoge.head()

,Unnamed: 0,ido,keido,mf_areasize,mf_area_sq,mf_r,mf_l,mf_d,mf_k,mf_s,...,dist_main_st0,dist_main_st1,dist_main_st2,dist_main_st3,dist_main_st4,dist_main_st5,dist_main_st6,shortest_mainst,pred_area,area_diff
0,0,35.748593,139.728979,20.01,400.4001,1,0,0,1,0,...,2680.392140,7042.774466,10379.576633,8252.841756,5817.141634,13401.130238,9529.072804,2680.392140,20.023508,-0.013508
1,1,35.662900,139.779212,16.50,272.2500,1,0,0,0,0,...,1926.998496,7728.405997,7037.280149,2315.401095,5709.344063,5299.058734,1926.998496,1926.998496,16.369177,0.130823
2,2,35.675148,139.667012,22.05,486.2025,1,0,0,1,0,...,3427.403872,3427.403872,3665.353210,9078.094214,10884.840336,8313.782950,8316.630133,3427.403872,22.081244,-0.031244
3,3,35.700200,139.651335,60.48,3657.8304,2,1,1,1,0,...,4585.343751,4585.343751,6536.126981,10679.449131,11501.726366,11237.229630,10382.063976,4585.343751,60.609699,-0.129699
4,4,35.766807,139.875451,39.66,1572.9156,2,0,1,1,0,...,10621.303770,17996.509046,19835.280580,13659.408140,10621.303770,19744.564094,15395.435610,10621.303770,39.588711,0.071289


In [5]:
hoge["pred_area"].head()

0    20.023508
1    16.369177
2    22.081244
3    60.609699
4    39.588711
Name: pred_area, dtype: float32

In [7]:
hoge["area_diff"].max()

3.9455108642578125

In [810]:
crazy_pred = crazy["賃料"].median()
print(crazy_pred)
pred = pd.DataFrame([crazy_pred for i in range(len(crazy.index))])
print(np.sqrt(mean_squared_error(crazy["賃料"],pred)))

650000.0
301848.99189789325


In [59]:
grid_params2={
    "xgb__n_estimators":[100,120,140],
    "xgb__max_depth":[7,8,9],
    "xgb__min_child_weight":[0,5,15],
    "xgb__learning_rate":[0.1,0.2],
    "xgb__random_state":[7777]
}
GCV2 = GridSearchCV(
    Pipeline(steps=rich_step),
    grid_params2,
    cv = 3,
    scoring="neg_mean_squared_error",
    verbose=3
)
GCV2.fit(temp_x_rich,temp_y_rich)

Fitting 3 folds for each of 54 candidates, totalling 162 fits
[CV] xgb__learning_rate=0.1, xgb__max_depth=7, xgb__min_child_weight=0, xgb__n_estimators=100, xgb__random_state=7777 
[17:05:14] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  xgb__learning_rate=0.1, xgb__max_depth=7, xgb__min_child_weight=0, xgb__n_estimators=100, xgb__random_state=7777, score=-20533275032.002, total=   0.6s
[CV] xgb__learning_rate=0.1, xgb__max_depth=7, xgb__min_child_weight=0, xgb__n_estimators=100, xgb__random_state=7777 
[17:05:14] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s


[CV]  xgb__learning_rate=0.1, xgb__max_depth=7, xgb__min_child_weight=0, xgb__n_estimators=100, xgb__random_state=7777, score=-15907774518.319, total=   0.6s
[CV] xgb__learning_rate=0.1, xgb__max_depth=7, xgb__min_child_weight=0, xgb__n_estimators=100, xgb__random_state=7777 
[17:05:15] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.3s remaining:    0.0s


[CV]  xgb__learning_rate=0.1, xgb__max_depth=7, xgb__min_child_weight=0, xgb__n_estimators=100, xgb__random_state=7777, score=-9381239329.321, total=   0.6s
[CV] xgb__learning_rate=0.1, xgb__max_depth=7, xgb__min_child_weight=0, xgb__n_estimators=120, xgb__random_state=7777 
[17:05:16] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[CV]  xgb__learning_rate=0.1, xgb__max_depth=7, xgb__min_child_weight=0, xgb__n_estimators=120, xgb__random_state=7777, score=-20454789224.898, total=   0.7s
[CV] xgb__learning_rate=0.1, xgb__max_depth=7, xgb__min_child_weight=0, xgb__n_estimators=120, xgb__random_state=7777 
[17:05:16] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[CV]  xgb__learning_rate=0.1, xgb__max_depth=7, xgb__min_child_weight=0, xgb__n_estimators=120, xgb__random_state=7777, score=-15826305572.156, total=   0.9s
[CV] xgb__learning_rate=0.1, xgb__max_depth=7, xgb__min_c

[Parallel(n_jobs=1)]: Done 162 out of 162 | elapsed:  2.2min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('drop_id',
                                        <my_class.drop_id object at 0x134b3b690>),
                                       ('parse_area',
                                        <my_class.parse_area_size object at 0x136792550>),
                                       ('parse_room',
                                        <my_class.parse_rooms object at 0x136792a50>),
                                       ('parse_old',
                                        <my_class.parse_how_old object at 0x136792990>),
                                       ('str_label',
                                        <my_class.structure_lab...
                                                     seed=None, silent=None,
                                                     subsample=1,
                                                     verbosity=1))],
         